# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

Импортируем все необходимые библиотеки

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Сохраняем файлы в переменные и выведем на экран

In [3]:
geo_data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
geo_data_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [5]:
geo_data_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Посмотрим на общую информацию

In [6]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В данных отсутствуют пропуски. Типы данных соответствуют содержимому

In [9]:
geo_data_0['product'].describe()

count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64

In [10]:
geo_data_1['product'].describe()

count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64

In [11]:
geo_data_2['product'].describe()

count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64

Убедились что в колонке **product** отсутствуют аномальные (отрицательные) значения

Природа данных в колонках **f0, f1, f2** не объясняется, соответственно не подлежат логическому анализу

Колонку **id** (уникальный идентификатор скважины), можно удалить. Она не несет в себе  информативности (в условиях данной задачи)

In [12]:
# удаление колонки id
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

Приступаем к следующему шагу

## Обучение и проверка модели

Разделим данные на признаки и целевой признак для каждого файла

In [13]:
data_0_features = geo_data_0.drop(['product'], axis=1)
data_0_target = geo_data_0['product']

data_1_features = geo_data_1.drop(['product'], axis=1)
data_1_target = geo_data_1['product']

data_2_features = geo_data_2.drop(['product'], axis=1)
data_2_target = geo_data_2['product']

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25

In [14]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    data_0_features, data_0_target, test_size=0.25, random_state=333)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    data_1_features, data_1_target, test_size=0.25, random_state=333)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    data_2_features, data_2_target, test_size=0.25, random_state=333)


In [15]:
print(features_train_0.shape, features_valid_0.shape, target_train_0.shape, target_valid_0.shape)
print(features_train_1.shape, features_valid_1.shape, target_train_1.shape, target_valid_1.shape)
print(features_train_2.shape, features_valid_2.shape, target_train_2.shape, target_valid_2.shape)

(75000, 3) (25000, 3) (75000,) (25000,)
(75000, 3) (25000, 3) (75000,) (25000,)
(75000, 3) (25000, 3) (75000,) (25000,)


Убедились что данные разбиты корректно 

Приступим к обучению модели. Для обучения будем использовать линейную регрессию (указано в условиях задачи)

In [16]:
array = [[features_train_0, target_train_0, features_valid_0, target_valid_0, 0], 
         [features_train_1, target_train_1, features_valid_1, target_valid_1, 1], 
         [features_train_2, target_train_2, features_valid_2, target_valid_2, 2]]

predicts = []

for region_data in array:
    model = LinearRegression()
    model.fit(region_data[0], region_data[1])
    predict = model.predict(region_data[2])
    rmse = mean_squared_error(region_data[3], predict) ** 0.5
    print('Регион: ', region_data[-1])
    print('RMSE: ', rmse)
    print('средний запас предсказанного сырья: ', predict.mean())
    print()
    predicts.append([rmse, predict])

Регион:  0
RMSE:  37.596877500413
средний запас предсказанного сырья:  92.25413811475417

Регион:  1
RMSE:  0.8847944517244747
средний запас предсказанного сырья:  68.4687776046565

Регион:  2
RMSE:  39.844211967707004
средний запас предсказанного сырья:  95.12352664262355



Сохраним предсказания в переменные 

In [17]:
predict_0 = predicts[0][-1]
predict_1 = predicts[1][-1]
predict_2 = predicts[2][-1]

Переведем предсказания в **Series**

In [18]:
predict_0 = pd.Series(predict_0)
predict_1 = pd.Series(predict_1)
predict_2 = pd.Series(predict_2)

Показатель RMSE в **0** и во **2** регионах в районе *37* и *39* соответственно. Показатель RMSE в **регионе-1** составляет: *0.88*. Предсказания модели в регионе-1 на порядок качественнее чем в двух других

Средний запас предсказанного сырья по регионам составляет:
- 0:  **92.254**
- 1:  **68.469**
- 2:  **95.124**

## Подготовка к расчёту прибыли

**Условия задачи:**

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают **200** лучших для разработки
- Бюджет на разработку скважин в регионе — **10 млрд рублей**
- Доход с каждой единицы продукта составляет — **450 тыс. рублей**

In [19]:
BUDGET = 10_000_000_000
INCOME_PER_UNIT = 450_000
TOP_B_HOLES = 200
SAMPLE_B_HOLES = 500

Посчитаем доход за единицу продукта со всех (200) скважин

In [20]:
total_income = INCOME_PER_UNIT * TOP_B_HOLES
total_income

90000000

Доход за единицу продукта со всех скважин составил - **90 млн. рублей**

Чтобы получить минимальный объем добычи для покрытия бюджета нужно поделить бюджет на доход за единицу

In [21]:
product = BUDGET / total_income
product

111.11111111111111

Объем сырья, необходимый для покрытия бюджета составляет - **111.111** 

Средний объем сырья предсказанный моделями составляет:
- 0:  **92.254**
- 1:  **68.469**
- 2:  **95.124**

Необходимый порог для безубыточной разработки привышает предсказания добычи по всем регионам

## Расчёт прибыли и рисков 

Создадим функцию для подсчета прибыли

Функция принимает в себя случайных 500 скважин с предсказанными объемами добычи и регион (0, 1, 2) берет лучшие 200 результата и возвращает общую прибыль

In [22]:
def profit_calc(predict_500, region):
    if region == 0:
        buffer_target = target_valid_0.reset_index(drop=True)
        buffer_predict_200 = predict_500.sort_values(ascending=False).head(TOP_B_HOLES)
        top_200 = buffer_target[buffer_predict_200.index]
    elif region == 1:
        buffer_target = target_valid_1.reset_index(drop=True)
        buffer_predict_200 = predict_500.sort_values(ascending=False).head(TOP_B_HOLES)
        top_200 = buffer_target[buffer_predict_200.index]
    elif region == 2:
        buffer_target = target_valid_2.reset_index(drop=True)
        buffer_predict_200 = predict_500.sort_values(ascending=False).head(TOP_B_HOLES)
        top_200 = buffer_target[buffer_predict_200.index]
    return (top_200.sum() * INCOME_PER_UNIT) - BUDGET

с помощью **.sample** получим случайные 500 скважин

In [23]:
sample_500_0 = predict_0.sample(n=500, replace=False, random_state=333)
sample_500_1 = predict_1.sample(n=500, replace=False, random_state=333)
sample_500_2 = predict_2.sample(n=500, replace=False, random_state=333)

In [24]:
profit_calc(sample_500_0, 0)

24494621.089157104

In [25]:
profit_calc(sample_500_1, 1)

407773681.274168

In [26]:
profit_calc(sample_500_2, 2)

643525168.1565533

Из случайной выборки (500) лучшие 200 месторождений приносят прибыль:
- в регионе - 0: **24,5 млн.**
- в регионе - 1: **407,7 млн.**
- в регионе - 2: **643,5 млн .**

Это лишь одна случайная выборка

Чтобы дать более качественную оценку предполагаемого дохода воспользуемся техникой Bootstrap

In [27]:
def risk_calc(predict, region):
    
    state = RandomState(333)
    total_income = [] # переменная хранит все рассчитанную прибыль всех 1000 итераций
    
    for i in range(1000):
        subsample = predict.sample(n=SAMPLE_B_HOLES, replace=True, random_state=state) # выборка из 500 строк
        profit = profit_calc(subsample, region) # обращаемся к ранее созданной функции для подсчета прибыли
        total_income.append(profit / 1_000_000) # делим полученную прибыль/убыток на миллион для читабельности и сохр.
        
    total_income = pd.Series(total_income) # переводим list в Series для получения Квантилей 
    
    lower = total_income.quantile(0.025) # левый квантиль
    upper = total_income.quantile(0.975) # правый квантиль
    
    print(f'Риск убытка регион-{region}: {(total_income < 0).mean() * 100}%')   
    print(f'Средняя прибыль: {total_income.mean():.2f} млн.') # средняя прибыль с 2 знаками после запятой 
    print(f'Доверительный интервал: {lower:.2f} | {upper:.2f}') 

In [28]:
risk_calc(predict_0, 0)

Риск убытка регион-0: 5.800000000000001%
Средняя прибыль: 402.82 млн.
Доверительный интервал: -107.16 | 870.61


In [29]:
risk_calc(predict_1, 1)

Риск убытка регион-1: 2.1%
Средняя прибыль: 424.76 млн.
Доверительный интервал: 21.61 | 798.09


In [30]:
risk_calc(predict_2, 2)

Риск убытка регион-2: 8.5%
Средняя прибыль: 346.97 млн.
Доверительный интервал: -188.57 | 818.91


Подведем итоги по каждому региону



                        Регион - 0 | Регион - 1 | Регион - 2 | 
      Средняя прибыль: |402.82 млн | 424.76 млн | 346.97 млн |
                 Риск: |    5.8%   |    2.1%    |    8.5%    |
                 RMSE: |   37.60   |    0.88    |    39.84   |

## Вывод

Лучший показатель RMSE получился у региона - 1. Точность прогноза в сравнении с остальными видна невооруженным глазом. Из результатов полученных в ходе исследования худшим регионом для добычи стал Регион-2 с самым высоким показателем риска в 8.5% и самым низким показателем средней прибыли.

Я бы советовал Регион - 1. У него самая высокая средняя прибыль 424.76 млн. и самый низкий показатель риска 2.1%